# Maintenance Notebook: Keeping the staggered_grid_config.yaml up to date

Progress is hindered by a weird problem where I cant replace the datafram in the esm store anymore, like I used to. I raised an [issue](https://github.com/NCAR/intake-esm/issues/246) on github.

## TODO: 

- One of the MPI versions has two different shift conventions (I might need to implement a 'force override' option.

In [1]:
# ! pip install git+https://github.com/NCAR/intake-esm.git
# ! pip install git+https://github.com/jbusecke/xmip.git

In [2]:
import xmip
xmip.__version__

'earthcube_commit+188.gf918540.dirty'

In [3]:
%load_ext autoreload
%autoreload 2
import intake
import pandas as pd
import xarray as xr
from xmip.grids import detect_shift
from xmip.preprocessing import combined_preprocessing

In [8]:
# url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
# col = intake.open_esm_datastore(url)

from xmip.utils import google_cmip_col
col = google_cmip_col()

<ipython-input-8-ddf6b9ba7c8c>:5: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  col = google_cmip_col()


In [9]:
col

,unique
activity_id,17
institution_id,35
source_id,84
experiment_id,160
member_id,549
table_id,37
variable_id,707
grid_label,10
zstore,410094
dcpp_init_year,60


In [10]:
# Grab all available ocean output.
def available_output(variables=None):
    """Show which source_id/grid_label combos have any data, and return a df that picks only one dataset for each combo"""
    url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
    col = intake.open_esm_datastore(url)
    query = dict(table_id=['Omon', 'Oyr'])
    if variables is not None:
        query['variable_id'] = variables
        
    
    # pick all available ocean fields for now
    cat = col.search(**query)
    print(cat)
    
    available = []
    dataframes = []
    df = cat.df.copy()
    groups = df.groupby(['source_id', 'grid_label'])
    for group in groups:
        
        # add source_id/grid_label combo to list
        label = '.'.join(group[0])
        # pick only the first index of each group
        line = group[1].iloc[0,:]
        
        available.append(label)
        dataframes.append(line)

    new_df = pd.concat(dataframes, axis=1).transpose()
    cat.df = new_df
    
    return cat, available

_, all_models = available_output()
print(len(all_models))

cat_tracer, tracer_models = available_output(['tos', 'thetao'])
missing_tracer_models = set(tracer_models).symmetric_difference(set(all_models))
print(f"Did not find tracer data for these models:{missing_tracer_models}\n")

cat_u, u_models = available_output(['uo'])
missing_u_models = set(u_models).symmetric_difference(set(all_models))
print(f"Did not find u data for these models:{missing_u_models}\n")

cat_v, v_models = available_output(['vo'])
missing_v_models = set(v_models).symmetric_difference(set(all_models))
print(f"Did not find v data for these models:{missing_v_models}\n")

print(f"Any models that have only u or v:{set(v_models).symmetric_difference(set(u_models))}")

<ipython-input-10-64c6a66bc5eb>:34: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  _, all_models = available_output()


<pangeo-cmip6 catalog with 1161 dataset(s) from 97437 asset(s)>
103


<ipython-input-10-64c6a66bc5eb>:37: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  cat_tracer, tracer_models = available_output(['tos', 'thetao'])


<pangeo-cmip6 catalog with 659 dataset(s) from 10852 asset(s)>
Did not find tracer data for these models:{'IPSL-CM6A-LR-INCA.gn', 'NorESM2-MM.grz', 'IPSL-CM6A-LR.gr', 'MRI-ESM2-0.gr2z', 'UKESM1-0-LL.gnz', 'HadGEM3-GC31-MM.gm', 'NorESM1-F.gr', 'MRI-ESM2-0.gm', 'NorCPM1.grz', 'UKESM1-0-LL.gm', 'NorESM2-LM.gm', 'MIROC6.gr', 'MRI-ESM2-0.gnz', 'EC-Earth3-CC.gn', 'MIROC6.gm', 'HadGEM3-GC31-LL.gm', 'HadGEM3-GC31-LL.gnz', 'NorESM2-LM.grz', 'NorCPM1.gm', 'IPSL-CM5A2-INCA.gn', 'NorESM2-MM.gm'}



<ipython-input-10-64c6a66bc5eb>:41: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  cat_u, u_models = available_output(['uo'])


<pangeo-cmip6 catalog with 365 dataset(s) from 1312 asset(s)>
Did not find u data for these models:{'CNRM-CM6-1.gr1', 'IPSL-CM6A-LR-INCA.gn', 'NorESM2-MM.grz', 'IPSL-CM6A-LR.gr', 'MRI-ESM2-0.gr2z', 'UKESM1-0-LL.gnz', 'CNRM-ESM2-1.gr1', 'CESM2-WACCM.gr', 'HadGEM3-GC31-MM.gm', 'NorCPM1.gn', 'NorESM1-F.gr', 'GFDL-ESM4.gn', 'MRI-ESM2-0.gm', 'CESM2.gr', 'EC-Earth3-Veg-LR.gn', 'CAS-ESM2-0.gn', 'MRI-ESM2-0.gn', 'EC-Earth3-Veg.gr', 'GFDL-OM4p5B.gn', 'NorCPM1.grz', 'EC-Earth3.gr', 'UKESM1-0-LL.gm', 'GFDL-OM4p5B.gr', 'GFDL-CM4.gr', 'EC-Earth3-AerChem.gn', 'NorESM2-LM.gm', 'MIROC6.gr', 'MRI-ESM2-0.gnz', 'CESM2-WACCM-FV2.gr', 'EC-Earth3-CC.gn', 'MIROC6.gm', 'GFDL-ESM4.gr', 'CESM2-FV2.gr', 'KACE-1-0-G.gr', 'HadGEM3-GC31-LL.gm', 'HadGEM3-GC31-LL.gnz', 'NorESM2-LM.grz', 'NorESM1-F.gn', 'CMCC-CM2-HR4.gn', 'GFDL-ESM2M.gn', 'CESM1-1-CAM5-CMIP5.gn', 'NorCPM1.gm', 'GISS-E2-1-H.gn', 'MRI-ESM2-0.gr', 'CMCC-ESM2.gn', 'IPSL-CM5A2-INCA.gn', 'NorESM2-MM.gm', 'CESM1-1-CAM5-CMIP5.gr', 'KIOST-ESM.gr1'}



<ipython-input-10-64c6a66bc5eb>:45: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  cat_v, v_models = available_output(['vo'])


<pangeo-cmip6 catalog with 375 dataset(s) from 1316 asset(s)>
Did not find v data for these models:{'CNRM-CM6-1.gr1', 'IPSL-CM6A-LR-INCA.gn', 'NorESM2-MM.grz', 'IPSL-CM6A-LR.gr', 'MRI-ESM2-0.gr2z', 'UKESM1-0-LL.gnz', 'CNRM-ESM2-1.gr1', 'CESM2-WACCM.gr', 'HadGEM3-GC31-MM.gm', 'NorCPM1.gn', 'NorESM1-F.gr', 'GFDL-ESM4.gn', 'MRI-ESM2-0.gm', 'CESM2.gr', 'CAS-ESM2-0.gn', 'MRI-ESM2-0.gn', 'EC-Earth3-Veg.gr', 'GFDL-OM4p5B.gn', 'NorCPM1.grz', 'EC-Earth3.gr', 'UKESM1-0-LL.gm', 'GFDL-OM4p5B.gr', 'GFDL-CM4.gr', 'NorESM2-LM.gm', 'MIROC6.gr', 'MRI-ESM2-0.gnz', 'CESM2-WACCM-FV2.gr', 'EC-Earth3-CC.gn', 'MIROC6.gm', 'GFDL-ESM4.gr', 'CESM2-FV2.gr', 'KACE-1-0-G.gr', 'HadGEM3-GC31-LL.gm', 'HadGEM3-GC31-LL.gnz', 'NorESM2-LM.grz', 'NorESM1-F.gn', 'CMCC-CM2-HR4.gn', 'GFDL-ESM2M.gn', 'CESM1-1-CAM5-CMIP5.gn', 'NorCPM1.gm', 'GISS-E2-1-H.gn', 'MRI-ESM2-0.gr', 'CMCC-ESM2.gn', 'IPSL-CM5A2-INCA.gn', 'NorESM2-MM.gm', 'CESM1-1-CAM5-CMIP5.gr', 'KIOST-ESM.gr1'}

Any models that have only u or v:{'EC-Earth3-AerChem.gn',

In [11]:
# for now load them manually
import fsspec
import xarray as xr
super_dict = {}
for var, cat in zip(['thetao', 'uo', 'vo'],[cat_tracer, cat_u, cat_v]):
    super_dict[var]={}
    for ri,(rr,row) in enumerate(cat.df.iterrows()):
#         print(ri)
        ds = combined_preprocessing(xr.open_zarr(fsspec.get_mapper(row['zstore']), consolidated=True, decode_times=False))
        label = f"{row['source_id']}.{row['grid_label']}"
        super_dict[var][label] = ds

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CESM2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CESM2-FV2: Unexpected unit (centimeters) for coordinate `lev` detected.
	 Converted to `m`


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


...

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/note

MIROC-ES2L: No units found


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


MIROC-ES2L: No units found
MIROC6: No units found


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/note

In [12]:
staggered_grid_dict = {}
for k in super_dict['thetao'].keys():
    ds_ref = super_dict['thetao'][k]
    s_id = ds_ref.attrs['source_id']
    g_la = ds_ref.attrs['grid_label']
    
    if not ('AWI' in k and 'gn' in k):
        print(f"############### {k} #######################")
        if k in super_dict['uo'].keys() and k in super_dict['vo'].keys():
            
            ds_u = super_dict['uo'][k]
            ds_v = super_dict['vo'][k]
            
            if 'x' not in ds_ref.dims:
                print(f'THIS IS SOME ERROR IN THE PREPROCESSSING. INVESTIGATE {k}')
                # a nevermind, these are just the AWI ones...remove them earlier...
            else:
                x_shift_u = detect_shift(ds_ref, ds_u, 'X')
                y_shift_u = detect_shift(ds_ref, ds_u, 'Y')

                x_shift_v = detect_shift(ds_ref, ds_v, 'X')
                y_shift_v = detect_shift(ds_ref, ds_v, 'Y')
                
                

                # check that there is only one left after removing 'center'
                x_shift = set([x_shift_u, x_shift_v]) - set(['center'])
                y_shift = set([y_shift_u, y_shift_v]) - set(['center'])
                # if they are all on center default to left

                if len(x_shift) == 0:
                    x_shift = 'left'
                elif len(x_shift) == 1:
                    x_shift = list(x_shift)[0]
                else:
                    print('SCHEISSE X')
                    print('x')
                    print(x_shift_u)
                    print(x_shift_v)
                    print('y')
                    print(y_shift_u)
                    print(y_shift_v)
                    x_shift=None

                if len(y_shift) == 0:
                    y_shift = 'left'
                elif len(y_shift) == 1:
                    y_shift = list(y_shift)[0]
                else:
                    print('SCHEISSE Y')
                    print('u')
                    print(x_shift_u)
                    print(y_shift_u)
                    print('v')
                    print(x_shift_v)
                    print(y_shift_v)
                    y_shift = None
        else:
            print(f"ATTENTION: Setting shift to left for {k}, since there is no velocity data")
            x_shift = 'left'
            y_shift = 'left'
            
        if x_shift is not None and y_shift is not None:
            if not s_id in staggered_grid_dict.keys():
                staggered_grid_dict[s_id] = {}
            staggered_grid_dict[s_id][g_la] = {'axis_shift':{'X': x_shift, 'Y': y_shift}}

############### ACCESS-CM2.gn #######################
############### ACCESS-ESM1-5.gn #######################
############### BCC-CSM2-MR.gn #######################
############### BCC-ESM1.gn #######################
############### CAMS-CSM1-0.gn #######################
############### CAS-ESM2-0.gn #######################
ATTENTION: Setting shift to left for CAS-ESM2-0.gn, since there is no velocity data
############### CESM1-1-CAM5-CMIP5.gn #######################
ATTENTION: Setting shift to left for CESM1-1-CAM5-CMIP5.gn, since there is no velocity data
############### CESM1-1-CAM5-CMIP5.gr #######################
ATTENTION: Setting shift to left for CESM1-1-CAM5-CMIP5.gr, since there is no velocity data
############### CESM2.gn #######################
############### CESM2.gr #######################
ATTENTION: Setting shift to left for CESM2.gr, since there is no velocity data
############### CESM2-FV2.gn #######################
############### CESM2-FV2.gr ######################

## Check with old dict...

In [13]:
import yaml
ff = open('/home/jovyan/xmip/xmip/specs/staggered_grid_config.yaml', "r")
grid_dict = yaml.safe_load(ff)
ff.close()

In [14]:
# any keys in the old dict that are not in the new one?
print(f"Keys in the old grid, which are not in the new one {set(grid_dict.keys())- set(staggered_grid_dict.keys())}")

print(f"Newly added models {set(staggered_grid_dict.keys()) - set(grid_dict.keys())}")

Keys in the old grid, which are not in the new one {'MPI-ESM1-2-LR'}
Newly added models {'EC-Earth3-AerChem', 'CMCC-CM2-HR4', 'CMCC-ESM2', 'KIOST-ESM', 'GFDL-ESM2M'}


In [17]:
staggered_grid_dict['GFDL-CM4']

{'gn': {'axis_shift': {'X': 'left', 'Y': 'left'}},
 'gr': {'axis_shift': {'X': 'left', 'Y': 'left'}}}

In [16]:
with open('test.yaml', 'w') as file:
    documents = yaml.dump(staggered_grid_dict, file)